## KNN Vecinos

Importamos las librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import ast 


Creamos los dos Dataframes

In [2]:
df1 = pd.read_csv("/workspaces/Kvecinos/data/raw/tmdb_5000_credits.csv")
df2 = pd.read_csv("/workspaces/Kvecinos/data/raw/tmdb_5000_movies.csv")

Los unimos para evitar todo el tema de trastear con SQL

In [3]:
joint= pd.merge(df1,df2,on='title')

Comprobamos

In [4]:
joint.head()

,movie_id,title,cast,crew,budget,genres,homepage,id,keywords,original_language,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,6.3,4466
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,7.6,9106
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",6.1,2124


Seleccionamos las columnas que nos interesan

In [5]:
df = joint[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [6]:
df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Comprobamos que no hay datos duplicados

In [7]:
df.duplicated().sum()

np.int64(0)

Creamos una función para extraer los datos

In [8]:
def extract_names(column):
    return column.apply(lambda x: [item["name"] for item in ast.literal_eval(x)])

df.loc[:, "genres_clean"] = extract_names(df["genres"])
df.loc[:, "keywords_clean"] = extract_names(df["keywords"])
df.loc[:, "crew_clean"] = extract_names(df["crew"])
df.loc[:, "cast_clean"] = extract_names(df["cast"])


/tmp/ipykernel_544/2303455746.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "genres_clean"] = extract_names(df["genres"])
/tmp/ipykernel_544/2303455746.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "keywords_clean"] = extract_names(df["keywords"])
/tmp/ipykernel_544/2303455746.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

### Convertir los datos a string

In [9]:
df.loc[:, "genres"] = df["genres_clean"]
df.loc[:, "keywords"] = df["keywords_clean"]
df.loc[:, "crew"] = df["crew_clean"]
df.loc[:, "cast"] = df["cast_clean"]

Unir todas las columnas en una única string y eliminar las comas

In [12]:
df['overview']=df['overview'].apply(lambda x: [str(x)])
df['genres']=df['genres'].apply(lambda x :[str(genre) for genre in x])
df['keywords']=df['keywords'].apply(lambda x :[str(keywords) for keywords in x])
df['crew']=df['crew'].apply(lambda x :[str(crew) for crew in x])
df['cast']=df['cast'].apply(lambda x :[str(cast) for cast in x])

/tmp/ipykernel_544/319233663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview']=df['overview'].apply(lambda x: [str(x)])
/tmp/ipykernel_544/319233663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres']=df['genres'].apply(lambda x :[str(genre) for genre in x])
/tmp/ipykernel_544/319233663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

Eliminamos las comas

In [11]:
df["tags"] = df["overview"] + df["genres"] + df["keywords"] + df["cast"] + df["crew"]
df["tags"] = df["tags"].apply(lambda x: ",".join(x).replace(",", " "))

/tmp/ipykernel_544/1260443092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tags"] = df["overview"] + df["genres"] + df["keywords"] + df["cast"] + df["crew"]
/tmp/ipykernel_544/1260443092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tags"] = df["tags"].apply(lambda x: ",".join(x).replace(",", " "))


In [13]:
df.iloc[0].tags

"In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Giovanni Ribisi Joel David Moore CCH Pounder Wes Studi Laz Alonso Dileep Rao Matt Gerald Sean Anthony Moran Jason Whyte Scott Lawrence Kelly Kilgour James Patrick Pitt Sean Patrick Murphy Peter Dillon Kevin Dorman Kelson Henderson David Van Horn Jacob Tomuri Michael Blain-Rozgay Jon Curry Luke Hawker Woody Schultz Peter Mensah Sonia Yee Jahnel Curfman Ilram Choi Kyla Warren Lisa Roumain Debra Wilson Chris Mala Taylor Kibby Jodie Landau Julie Lamm Cullen B. Madden Joseph Brady Madden Frankie Torres Austin 

Convertir texto en matriz numérica

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["tags"])

model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

### Creamos una función para recomendar pelis en base a la distancia. 

En un principio lo hice a la manera del ejercicio propuesta, pero quise complicarme un poco. 

El código al final no es mío, me ha ayudado la IA, pero quería entender o al menos ver, cual era la solución propuesta. 

In [18]:
def recomiendapeliculas_por_titulo(titulo_peli):
    try:
        indice_peli = df[df["title"] == titulo_peli].index[0]
    except IndexError:
        return f"No se encontró la película con el título '{titulo_peli}'."
    distances, indices = model.kneighbors(tfidf_matrix[indice_peli])
    peli_similar = [(df["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return peli_similar[1:]  #Este código no incluye la pelicula original

def recomiendapeliculas_por_genero(genero_peli):
    mask = df["genres"].apply(lambda x: genero_peli.lower() in [g.lower() for g in x]) #Usamos la terminología de mask para referirnos a filtros booleanos
    if not mask.any():
        return f"No se encontraron películas con el género '{genero_peli}'."
    indice_peli = mask[mask].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[indice_peli])
    peli_similar = [(df["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return peli_similar[1:]  #Este código no incluye la pelicula original

# Función de recomendación por actor
def recomiendapeliculas_por_actor(actor_peli):
    mask = df["cast"].apply(lambda x: actor_peli.lower() in [actor.lower() for actor in x])
    if not mask.any():
        return f"No se encontraron películas con el actor '{actor_peli}'."
    indice_peli = mask[mask].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[indice_peli])
    peli_similar = [(df["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return peli_similar[1:]  #Este código no incluye la pelicula original


Me ha parecido muy interesante conceptos como mask para el filtrado de datos. 

Me he dado cuenta, no obstante, que el DF no es tan diverso como uno espera, lo cual limita mucho las búsquedas. 

He creado una función que pregunta para personalizar el acceso a cada uno de los request. 

También es limitada porque hay que introducir el texto exacto. 

Pero algo es algo. 

In [24]:
def recomendacion_peliculas(eleccion, palabra):
    try:
        if eleccion == 'actor':
            indice_peli = df[df["cast"].apply(lambda x: palabra.lower() in [actor.lower() for actor in x])].index[0]
        elif eleccion == 'genero':
            indice_peli = df[df["genres"] == palabra].index[0]
        elif eleccion == 'titulo':
            indice_peli = df[df["title"] == palabra].index[0]
        else:
            return []

        distances, indices = model.kneighbors(tfidf_matrix[indice_peli])
        peli_similar = [(df["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
        return peli_similar[1:]  
    except IndexError:
        print(f"No se encontraron resultados para '{palabra}'.")
        return []

def cunado_peliculero():
    eleccion = input("¿Qué quieres buscar? (actor, genero, titulo): ").lower()

    if eleccion not in ['actor', 'genero', 'titulo']:
        print("Opción no válida")
        return

    palabra = input(f"Escribe el {eleccion}: ")

    recommendations = recomendacion_peliculas(eleccion, palabra)

    if recommendations:
        print(f"Film recommendations based on '{palabra}':")
        for peli, distance in recommendations:
            print(f"- Film: {peli}")
    else:
        print("No recommendations found.")

cunado_peliculero()

No se encontraron resultados para 'pirates'.
No recommendations found.
